# Week 10 - Stream Join Processing
Bounded Stream Join
- There is a start and end
Unbounded Stream Join
There is a start but no end

Due to network latency, some data arrives late
Data which matches each other might be missed for a join operation
Use 2 symmetric hash tables to avoid missing the join of any incoming tuple